In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# 20241014
- **D-4**

## 가설 설정
 - 주어진 변수를 활용하여 귀무가설과 대립가설을 설정하세요.

 - simple one-linear-Regression
  + Y = petal_width , X = sepal_length
  + H0: Y = b0 + b1X, & b1 = 0
  + H1: not H0

In [ ]:
!pip install pingouin -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pingouin as pg

In [ ]:
import seaborn as sns
import pandas as pd
from statsmodels.formula.api import ols

iris = sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


- one w/ pingouin, other w/ statmodels

In [ ]:
import statsmodels.api as sm

x = iris['sepal_length']
y = iris['petal_width']

X = sm.add_constant(x) # add dummie constant

result = ols('petal_width ~ sepal_length', data = iris).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            petal_width   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     299.2
Date:                Mon, 14 Oct 2024   Prob (F-statistic):           2.33e-37
Time:                        00:40:37   Log-Likelihood:                -88.686
No. Observations:                 150   AIC:                             181.4
Df Residuals:                     148   BIC:                             187.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.2002      0.257    -12.458   

In [ ]:
res = pg.linear_regression(x, y)
print(res)

          names      coef        se          T          pval        r2  \
0     Intercept -3.200215  0.256886 -12.457735  8.141394e-25  0.669028   
1  sepal_length  0.752918  0.043530  17.296454  2.325498e-37  0.669028   

     adj_r2  CI[2.5%]  CI[97.5%]  
0  0.666791 -3.707853  -2.692577  
1  0.666791  0.666897   0.838939  


 - 회귀 모델의 결정계수 -[adj r^2]

In [ ]:
result.rsquared_adj

0.6667913866278083

In [ ]:
res['adj_r2'][0]

0.6667913866278081

- 회귀계수 구하기

In [ ]:
result.params

,0
Intercept,-3.200215
sepal_length,0.752918


In [ ]:
res['coef']

,coef
0,-3.200215
1,0.752918


In [ ]:
# about slope
print(result.params[1],'\n',
res['coef'][1])

0.7529175706758456 
 0.7529175706758454


<ipython-input-20-dbe87a9fab3b>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(result.params[1],'\n',


In [ ]:
# abput constant
print(result.params[0],'\n',
res['coef'][0])

-3.2002150046491926 
 -3.2002150046491913


<ipython-input-22-419dc88329ad>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(result.params[0],'\n',


- p-value만 구하기

In [ ]:
print(result.pvalues[1],'\n',
res['pval'][1])

2.32549807979331e-37 
 2.3254980797956973e-37


<ipython-input-25-533b6b7dd7c0>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(result.pvalues[1],'\n',


In [ ]:
print(f'{result.pvalues[1]:.38f}')

0.00000000000000000000000000000000000023


<ipython-input-40-a38eeedb6574>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'{result.pvalues[1]:.38f}')


- Predict w/ model

In [ ]:
result_ols = result.params[1] * 6 + result.params[0]
res_pg = res['coef'][1] * 6 + res['coef'][0]
print(result_ols, res_pg)

1.317290419405881 1.3172904194058814


<ipython-input-26-30d69009d9d1>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result_ols = result.params[1] * 6 + result.params[0]


In [ ]:
new_data = pd.DataFrame({'sepal_length': [5,6,7,234,1143]})

result_mdl = result.predict(new_data)
print(result_mdl)

0      0.564373
1      1.317290
2      2.070208
3    172.982497
4    857.384568
dtype: float64


- 회귀모델의 SSE

In [ ]:
print(result.ssr, '\n',
      res['se'][0])

28.65225115414104 
 0.2568857904046287


In [ ]:
# dep.variable = petal_width, ind.variable = sepal_length

iris['residuals'] = iris['petal_width'] - result.predict(iris['sepal_length'])
iris['residuals'].head()

,residuals
0,-0.439665
1,-0.289081
2,-0.138498
3,-0.063206
4,-0.364373


In [ ]:
iris['predict'] = result.predict(iris['sepal_length'])
iris['predict'].head()

,predict
0,0.639665
1,0.489081
2,0.338498
3,0.263206
4,0.564373


- 회귀모델의 MSE

In [ ]:
print(result.mse_resid, '\n',
      res['se'][0])

0.19359629158203406 
 0.2568857904046287


In [ ]:
MSE = (iris['residuals']**2).mean()
print(MSE)

0.1910150076942736


- 신뢰구간 구하기 (.95)

In [ ]:
result.conf_int(0.05).loc['sepal_length']

,sepal_length
0,0.666897
1,0.838939


In [ ]:
# sepal_length 가 6일때 a = 0.05의 예측구간 구하기

new_data = pd.DataFrame({'sepal_length' : [6]})
pred = result.get_prediction(new_data)
pred.summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,1.31729,0.036567,1.245029,1.389551,0.444807,2.189774


# Pingouin 으로 statsmodels와 같은 결과 출력하기
 - 이 때 종속변수는 petal width, 독립변수는 sepal length 이다.

In [ ]:
res = pg.linear_regression(iris['sepal_length'], iris['petal_width'])
res

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-3.200215,0.256886,-12.457735,8.141394e-25,0.669028,0.666791,-3.707853,-2.692577
1,sepal_length,0.752918,0.043530,17.296454,2.325498e-37,0.669028,0.666791,0.666897,0.838939


- 회귀계수 R-squared

In [ ]:
res['r2'][1]

0.6690276860464135

- p-value

In [ ]:
res['pval'][1]

2.3254980797956973e-37

- predict w/ other values

In [ ]:
import numpy as np

new_data = np.array([6,7,12])

pred_val = res['coef'][1] * new_data + res['coef'][0]
print(pred_val)

[1.31729042 2.07020799 5.83479584]


- SSR

In [ ]:
iris['pred'] = pred_val = res['coef'][1] * iris['sepal_length'] + res['coef'][0]

iris['resid'] = iris["petal_width"] - iris['pred']

(iris['resid']**2).sum()

28.652251154141037

- MSE

In [ ]:
(iris['resid']**2).mean()

0.19101500769427357

In [ ]:
(iris['resid']**2).sum() / (len(iris['resid'] - 2))

0.19101500769427357

- a = 0.05, CI

In [ ]:
res['CI[2.5%]'][1], res['CI[97.5%]'][1]

(0.6668966227182522, 0.8389385186334386)

# 또 또 또 API 크롤링


- https://www.alphavantage.co/
- ACI47VTVRI0XYO1J

In [ ]:
import requests

url = 'https://openapi.shinhancard.com:8443/v1.0/shinhanplus/cards/cardlistinappcard'


{'data': [{'transaction_date': '2024-10-01', 'ticker': 'BA', 'executive': 'MOLLENKOPF, STEVEN M', 'executive_title': 'Director', 'security_type': 'Phantom Stock Units', 'acquisition_or_disposal': 'A', 'shares': '956.226', 'share_price': '0.0'}, {'transaction_date': '2024-10-01', 'ticker': 'BA', 'executive': 'BRADWAY, ROBERT A', 'executive_title': 'Director', 'security_type': 'Phantom Stock Units', 'acquisition_or_disposal': 'A', 'shares': '580.274', 'share_price': '0.0'}, {'transaction_date': '2024-10-01', 'ticker': 'BA', 'executive': 'DOUGHTIE, LYNNE M', 'executive_title': 'Director', 'security_type': 'Phantom Stock Units', 'acquisition_or_disposal': 'A', 'shares': '326.915', 'share_price': '0.0'}, {'transaction_date': '2024-10-01', 'ticker': 'BA', 'executive': 'GITLIN, DAVID L.', 'executive_title': 'Director', 'security_type': 'Phantom Stock Units', 'acquisition_or_disposal': 'A', 'shares': '547.582', 'share_price': '0.0'}, {'transaction_date': '2024-10-01', 'ticker': 'BA', 'executiv

In [312]:
import requests
import pandas as pd
import json

KEY_VALUE = 'HRYWiTp0KEDDEYYyUQV9msoe%2B2YK6PPbdhjgspJt5p%2F3YskNrgY3nyiVew4VxbwA6m4F5kRUXNZaEa2Z0kPFVg%3D%3D'
pageNo = 3
numOfRows = 10

url_1 = f'http://apis.data.go.kr/1160100/service/GetFnCoFinaStatCredInfoService_V2/getFnCoSummFinaStat_V2?pageNo={pageNo}&numOfRows={numOfRows}&resultType=json&fnccmpNm=%EA%B9%80%EC%B2%9C%EC%A0%80%EC%B6%95%EC%9D%80%ED%96%89&serviceKey={KEY_VALUE}' #요약재무제표
url_2 = f'http://apis.data.go.kr/1160100/service/GetFnCoFinaStatCredInfoService_V2/getFnCoBs_V2?pageNo={pageNo}&numOfRows={numOfRows}&resultType=json&fnccmpNm=%EA%B9%80%EC%B2%9C%EC%A0%80%EC%B6%95%EC%9D%80%ED%96%89&serviceKey={KEY_VALUE}' #재무상태표
url_3 = f"http://apis.data.go.kr/1160100/service/GetFnCoFinaStatCredInfoService_V2/getFnCoIs_V2?pageNo={pageNo}&numOfRows={numOfRows}&resultType=json&fnccmpNm=%EA%B9%80%EC%B2%9C%EC%A0%80%EC%B6%95%EC%9D%80%ED%96%89&serviceKey={KEY_VALUE}" #손익계산서

res_1 = requests.get(url_1).json()
res_2 = requests.get(url_2).json()
res_3 = requests.get(url_3).json()


In [313]:
dat_res_1 = pd.DataFrame(res_1['response']['body']['items']['item']) # max 724, 250, 124, 124
dat_res_2 = pd.DataFrame(res_2['response']['body']['items']['item']) # max 6255
dat_res_3 = pd.DataFrame(res_3['response']['body']['items']['item']) # max 2993

In [326]:
def read_finance_data(service_type_input, KEY_VALUE = 'HRYWiTp0KEDDEYYyUQV9msoe%2B2YK6PPbdhjgspJt5p%2F3YskNrgY3nyiVew4VxbwA6m4F5kRUXNZaEa2Z0kPFVg%3D%3D'):

    numOfRows = 0
    dat_res = None

    if service_type_input == 1:
      service_type = 'getFnCoSummFinaStat_V2'
      numOfRows = 750
    elif service_type_input == 2:
      service_type = 'getFnCoBs_V2'
      numOfRows = 6300
    elif service_type_input == 3:
      service_type = 'getFnCoIs_V2'
      numOfRows = 3000

    url = f'http://apis.data.go.kr/1160100/service/GetFnCoFinaStatCredInfoService_V2/{service_type}?pageNo=1&numOfRows={numOfRows}&resultType=json&fnccmpNm=%EA%B9%80%EC%B2%9C%EC%A0%80%EC%B6%95%EC%9D%80%ED%96%89&serviceKey={KEY_VALUE}'
    res = requests.get(url).json()
    dat_res = pd.concat([dat_res, pd.DataFrame(res['response']['body']['items']['item'])])

    return dat_res

In [327]:
dat_1 = read_finance_data(1)
dat_2 = read_finance_data(2)
dat_3 = read_finance_data(3)

In [331]:
dat_1.shape, dat_2.shape, dat_3.shape

((724, 17), (6255, 13), (2993, 13))

In [332]:
dat_1.head()

,basDt,crno,bizYear,rptCd,rptCdNm,fnclDcd,fnclDcdNm,fncoSaleAmt,fncoBzopPft,iclsPalClcAmt,fncoCrtmNpf,fncoTastAmt,enpTdbtAmt,fncoTcptAmt,fncoCptlAmt,fncoDebtRto,curCd
0,20150331,1101110514011,2015,,,ifrs_ConsolidatedMember,연결요약재무제표,18646233892,5998381178,5765631181,4974769000,12805835476,2392180699,10413654777,0,22.9715767444,KRW
1,20150331,1101110514011,2015,,,ifrs_SeparateMember,별도요약재무제표,15080755890,5390575989,5372236561,4582399215,10316417748,344353253,9972064495,0,3.4531791604,KRW
2,20150930,1101110035108,2015,,,ifrs_ConsolidatedMember,연결요약재무제표,161549573391,6498861171,8146779721,6671419865,259201088335,80529713225,178671375110,0,45.0714129084,KRW
3,20150930,1101110035108,2015,,,ifrs_SeparateMember,별도요약재무제표,125609417161,3351378651,7179835194,5698434840,240918688272,43356318050,197562370222,21157045000,21.9456357004,KRW
4,20151130,1243110000250,2015,,,ifrs_SeparateMember,별도요약재무제표,109829319986,1706488358,5032040373,1593782281,146537771265,42105380007,104432391258,0,40.3183145572,KRW
